In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14788596498408729037, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 10968950375
 locality {
   bus_id: 2
 }
 incarnation: 12346174965249874313
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:83:00.0, compute capability: 6.1", name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 10968950375
 locality {
   bus_id: 2
 }
 incarnation: 14811794599130846796
 physical_device_desc: "device: 1, name: GeForce GTX 1080 Ti, pci bus id: 0000:84:00.0, compute capability: 6.1", name: "/device:GPU:2"
 device_type: "GPU"
 memory_limit: 10968950375
 locality {
   bus_id: 2
 }
 incarnation: 267119966395715367
 physical_device_desc: "device: 2, name: GeForce GTX 1080 Ti, pci bus id: 0000:87:00.0, compute capability: 6.1", name: "/device:GPU:3"
 device_type: "GPU"
 memory_limit: 10968950375
 locality {
   bus_id: 2
 }
 incarnation: 4162486414971123552
 physical_devi

In [3]:
import keras
import tensorflow as tf
import keras.backend as K

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.4.0
dim_ordering: tf


Using TensorFlow backend.


In [4]:
import os
import numpy as np
import math
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Model
from keras.layers import Input, Activation, merge, Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
# from keras.layers.convolutional import Convolution2D, MaxPooling2D

from keras.optimizers import Adam, SGD, RMSprop
from keras.applications.inception_v3 import InceptionV3
from keras.regularizers import l2

from keras.utils import plot_model
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams["axes.grid"] = False
plt.rcParams.update({'font.size': 20})

In [5]:
train_dir = 'data-ISIC18/train/'
test_dir = 'data-ISIC18/test/'
test_without_aug_dir = 'data-ISIC18/test-without-aug/'

In [6]:
for root,dirs,files in os.walk(train_dir):
    print (root, len(files))

print("*"*30)
for root,dirs,files in os.walk(test_dir):
    print (root, len(files))
    
print("*"*30)
for root,dirs,files in os.walk(test_without_aug_dir):
    print (root, len(files))

data-ISIC18/train/ 0
data-ISIC18/train/DF 4152
data-ISIC18/train/VASC 3488
data-ISIC18/train/MEL 3308
data-ISIC18/train/AKIEC 3856
data-ISIC18/train/BKL 3275
data-ISIC18/train/BCC 4773
data-ISIC18/train/NV 4975
******************************
data-ISIC18/test/ 0
data-ISIC18/test/DF 1632
data-ISIC18/test/VASC 1484
data-ISIC18/test/MEL 1664
data-ISIC18/test/AKIEC 1552
data-ISIC18/test/BKL 1479
data-ISIC18/test/BCC 1692
data-ISIC18/test/NV 1730
******************************
data-ISIC18/test-without-aug/ 0
data-ISIC18/test-without-aug/DF 30
data-ISIC18/test-without-aug/VASC 33
data-ISIC18/test-without-aug/MEL 312
data-ISIC18/test-without-aug/AKIEC 74
data-ISIC18/test-without-aug/BKL 284
data-ISIC18/test-without-aug/BCC 139
data-ISIC18/test-without-aug/NV 1730


In [7]:
img_height, img_width = 300, 225
input_shape = (img_height, img_width, 3)
batch_size = 32
epochs = 50

In [8]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    validation_split= 0.25)
#     zoom_range=0.2,
#     shear_range=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'validation',
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

test_generator_without_aug = test_datagen.flow_from_directory(
    test_without_aug_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 20872 images belonging to 7 classes.
Found 6955 images belonging to 7 classes.
Found 11233 images belonging to 7 classes.
Found 2602 images belonging to 7 classes.


In [9]:
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
predict_size_train = int(math.ceil(nb_train_samples / batch_size))
num_classes = len(train_generator.class_indices)

print("nb_train_samples:", nb_train_samples)
print("\npredict_size_train:", predict_size_train)

print("\n num_classes:", num_classes)

nb_train_samples: 20872

predict_size_train: 653

 num_classes: 7


In [10]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=Input(shape=input_shape))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x)
prediction = Dense(num_classes, activation=tf.nn.softmax)(x)

model = Model(inputs=base_model.input,outputs=prediction)

In [11]:
adam_opt=Adam(lr = 0.0001, beta_1=0.6, beta_2=0.99, epsilon=None, decay=0.0, amsgrad=True)
sgd_opt = SGD(lr=1e-06, momentum=0.0, decay=0.0, nesterov=False)
rmsp_opt = RMSprop(lr=1e-4, decay=0.9)

model.compile(optimizer= adam_opt, loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit_generator(
  train_generator,
  steps_per_epoch = nb_train_samples // batch_size,
  epochs = epochs,
  validation_data = validation_generator,
  validation_steps = nb_validation_samples // batch_size,
  verbose = 2)

Epoch 1/50
 - 287s - loss: 2.5465 - acc: 0.1516 - val_loss: 2.4413 - val_acc: 0.1554
Epoch 2/50
